In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,summarize)
from ISLP import confusion_table
from ISLP.models import contrast
from sklearn.discriminant_analysis import (LinearDiscriminantAnalysis as LDA,QuadraticDiscriminantAnalysis as QDA)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import siuba
from siuba import _, if_else, select, mutate

In [14]:
smarket = load_data('Smarket')

In [15]:
smarket2=(
    smarket >>
    #mutate(Direction = if_else(_.Direction == 'Up',1,0)) >>
    mutate(train = if_else(_.Year < 2005,1,0) ) >>
    select(~_.Year,~_.Today)

)
# print(smarket2.head())
design = MS(smarket2 >> select(~_.Direction))
# print(design)

X = design.fit_transform(smarket2)
y = smarket2['Direction']  == 'Up'
train = (smarket2.train == 1)
X_train, X_test = X.loc[train], X.loc[~train]
y_train, y_test = y.loc[train], y.loc[~train]

In [8]:


# Create a logistic regression model
model = LogisticRegression()

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model's accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(pd.DataFrame({'y_test': y_test, 'y_pred':y_pred}))

Accuracy: 0.48412698412698413
      y_test  y_pred
998    False    True
999    False    True
1000   False    True
1001    True    True
1002   False   False
...      ...     ...
1245    True   False
1246   False    True
1247    True    True
1248   False    True
1249   False    True

[252 rows x 2 columns]


In [16]:
glm_train = sm.GLM(y_train,X_train,family=sm.families.Binomial())

In [62]:
probs = results.predict(exog=X_test)
probs= np.array(probs)
r1=pd.DataFrame({'prb':probs})
r1 = (
    r1

    >>  mutate(prd = if_else(_.prb>=0.5, True,False))
    >> mutate(real_value = np.array(y_test))
)


In [63]:
accuracy = accuracy_score(r1.prd, r1.real_value)
accuracy

0.4365079365079365

In [64]:
ct = confusion_table(r1.prd, r1.real_value)
print(ct)

Truth      False  True 
Predicted              
False        110    141
True           1      0
